In [1]:
import tensorflow as tf
import numpy as np
import os
import time
import datetime
from tensorflow.contrib import learn
import pandas as pd

import matplotlib.pyplot as plt
import matplotlib as mpl
%matplotlib inline

In [2]:
sess=tf.Session()    
#First let's load meta graph and restore weights
saver = tf.train.import_meta_graph('model-9500.meta')
saver.restore(sess,tf.train.latest_checkpoint('./'))

INFO:tensorflow:Restoring parameters from /Users/adb/Google Drive/Data Analytics/project/xtalkdb/Deep Learning/num_filter_64_fold_0/1512825696/checkpoints/model-9900


In [3]:
# Now, let's access and create placeholders variables and
# create feed-dict to feed new data
 
graph = tf.get_default_graph()
# w1 = graph.get_tensor_by_name("w1:0")
# w2 = graph.get_tensor_by_name("w2:0")
# feed_dict ={w1:13.0,w2:17.0}
 
# #Now, access the op that you want to run. 
# op_to_restore = graph.get_tensor_by_name("op_to_restore:0")
 
# print sess.run(op_to_restore,feed_dict)

In [4]:
from utils import *

In [110]:
# Load data
x_text, y = load_data_and_labels()

# Build vocabulary
max_document_length = max([len(x.split(" ")) for x in x_text])
vocab_processor = learn.preprocessing.VocabularyProcessor(max_document_length)
x = np.array(list(vocab_processor.fit_transform(x_text)))
print("Vocabulary Size: {:d}".format(len(vocab_processor.vocabulary_)))


encodedPathwayA, encodedPathwayB = list(vocab_processor.transform(['pathwayA pathwayB']))[0][:2]
print("encodedPathwayA = %s" % encodedPathwayA, "encodedPathwayB = %s" % encodedPathwayB)

word_distancesA = load_word_distancesA()
word_distancesB = load_word_distancesB()

pos_embedding = load_pos_embedding()

Vocabulary Size: 33447
encodedPathwayA = 8 encodedPathwayB = 53


In [111]:
sess.graph.get_operations()

[<tf.Operation 'input_x' type=Placeholder>,
 <tf.Operation 'input_y' type=Placeholder>,
 <tf.Operation 'dropout_keep_prob' type=Placeholder>,
 <tf.Operation 'word_distancesA' type=Placeholder>,
 <tf.Operation 'word_distancesB' type=Placeholder>,
 <tf.Operation 'encoded_pos' type=Placeholder>,
 <tf.Operation 'Const' type=Const>,
 <tf.Operation 'word_embedding/random_uniform/shape' type=Const>,
 <tf.Operation 'word_embedding/random_uniform/min' type=Const>,
 <tf.Operation 'word_embedding/random_uniform/max' type=Const>,
 <tf.Operation 'word_embedding/random_uniform/RandomUniform' type=RandomUniform>,
 <tf.Operation 'word_embedding/random_uniform/sub' type=Sub>,
 <tf.Operation 'word_embedding/random_uniform/mul' type=Mul>,
 <tf.Operation 'word_embedding/random_uniform' type=Add>,
 <tf.Operation 'word_embedding/W' type=VariableV2>,
 <tf.Operation 'word_embedding/W/Assign' type=Assign>,
 <tf.Operation 'word_embedding/W/read' type=Identity>,
 <tf.Operation 'word_embedding/embedding_lookup' t

In [113]:
input_x = sess.graph.get_tensor_by_name('input_x:0')
input_y = sess.graph.get_tensor_by_name('input_y:0')
dropout_keep_prob = sess.graph.get_tensor_by_name('dropout_keep_prob:0')

temp = sess.run(sess.graph.get_tensor_by_name('word_embedding/W/read:0'), feed_dict = {
              input_x: x[:500],
              input_y: y[:500],
              dropout_keep_prob: 1.0
            })

In [114]:
temp

array([[ 0.42573053, -0.57253414, -0.82029009, ...,  0.24011643,
        -0.66164088, -0.65945202],
       [-0.25906304, -0.21594091, -0.5241791 , ...,  0.8542459 ,
        -0.77184266, -0.05237999],
       [-0.70570719,  0.7973457 , -0.34505397, ...,  0.63056016,
         0.65847784, -0.75818044],
       ..., 
       [ 0.2594206 , -0.96879363,  0.06707843, ..., -0.97574419,
         0.45838261,  1.01281953],
       [ 0.86688876, -0.02289724,  0.13965821, ..., -0.38101697,
         0.18137598,  0.93196487],
       [-0.55013132,  0.49932289, -0.97040486, ..., -0.1171782 ,
        -0.84769702,  0.21874619]], dtype=float32)

In [115]:
a = np.matmul(temp, temp[8])


In [116]:
vocab_processor.vocabulary_.reverse(4082)

'antibodies'

In [117]:
for x in np.argsort(a/np.sum(a))[-10:]:
    print(vocab_processor.vocabulary_.reverse(x))

paracellular
aicar
initiation
tsh-induced
conclusions
tumorigenesis
cytotoxicity
go
long-term
antibodies


In [118]:
embeddings = temp

In [119]:
words = ['cell', 'signaling', 'activate', 'inhibit', 'crosstalk', 'bind', 'regulate', "phosporylate", "transcription", "pathway", "interact"]
ids = list(vocab_processor.transform([' '.join(words)]))[0][:len(words)]
ids

array([170,  26, 523, 259,  23, 830, 350,   0,  91, 151, 595])

In [120]:
X = embeddings[[ids]]

In [121]:
from sklearn.metrics.pairwise import cosine_similarity, euclidean_distances
dist = cosine_similarity(X, embeddings)

In [122]:
dist.shape

(11, 33447)

In [123]:
embeddings.shape

(33447, 128)

In [124]:
dist

array([[ 0.02268307, -0.03653714,  0.0446897 , ...,  0.05397145,
        -0.00484553, -0.0254921 ],
       [ 0.03520763,  0.01536767, -0.09859843, ...,  0.00190092,
        -0.00145259,  0.07821229],
       [ 0.02852749, -0.02158012,  0.06123535, ..., -0.03685302,
         0.00022058,  0.0300544 ],
       ..., 
       [-0.0466223 , -0.04053101,  0.17909929, ...,  0.04071163,
         0.04380332,  0.03429671],
       [-0.1005505 ,  0.01270755, -0.00687877, ..., -0.06773221,
         0.03864231,  0.10599624],
       [ 0.02097949,  0.08812883, -0.15234043, ...,  0.03715345,
         0.01511482, -0.01386776]], dtype=float32)

In [125]:
# prob = dist/np.sum(dist, axis=0)

In [126]:
dist.argsort(axis=1)

array([[23602, 29706,  1214, ..., 10761, 17411,   170],
       [ 3118,   266, 26270, ..., 19782, 28330,    26],
       [ 3687, 21849, 15825, ..., 23023, 15537,   523],
       ..., 
       [31094, 11646, 33383, ...,  5678, 26610,    91],
       [30083, 10290, 22800, ..., 27599,  4778,   151],
       [13054,  5859, 11588, ..., 30567, 30851,   595]])

In [127]:
dist.argsort(axis=1)[:,::-1]

array([[  170, 17411, 10761, ...,  1214, 29706, 23602],
       [   26, 28330, 19782, ..., 26270,   266,  3118],
       [  523, 15537, 23023, ..., 15825, 21849,  3687],
       ..., 
       [   91, 26610,  5678, ..., 33383, 11646, 31094],
       [  151,  4778, 27599, ..., 22800, 10290, 30083],
       [  595, 30851, 30567, ..., 11588,  5859, 13054]])

In [128]:
most_similar_words = dist.argsort(axis=1)[:,::-1][:,:20]

In [129]:
# most_similar_words = most_similar_words.transpose()

In [131]:
for i in range(len(words)):
    print('Top 20 similar words for %s are =>\n' % words[i].upper())
    mwords = [vocab_processor.vocabulary_.reverse(most_similar_words[i][j]) for j in range(6)]
    print(', '.join(mwords))
    print('\n\n') 

Top 20 similar words for CELL are =>

cell, 790, fy, steroidogenesis, pro-opiomelanocortin, 0096997



Top 20 similar words for SIGNALING are =>

signaling, 24126783, watkinson, undetectable, 250012, p53-mediated



Top 20 similar words for ACTIVATE are =>

activate, inositol-phosphorylceramide, trenerry, mimics, eomes, pacific



Top 20 similar words for INHIBIT are =>

inhibit, annotated, oist, ekman, 48109, 1742-4682-10-27



Top 20 similar words for CROSSTALK are =>

crosstalk, querétaro, least, betaine, 23020846, 2007-0870



Top 20 similar words for BIND are =>

bind, damage, q-rt-pcr, functioning, heat-labile, detection



Top 20 similar words for REGULATE are =>

regulate, luhr, 2165, eam, polish, baltimore



Top 20 similar words for PHOSPORYLATE are =>

<UNK>, 1108, ex-vivo, laboratoty, pgd2, 1189



Top 20 similar words for TRANSCRIPTION are =>

transcription, bases, nuclear-inducible, 1182, uhnres, eliciting



Top 20 similar words for PATHWAY are =>

pathway, pmc2837737, d

In [93]:
most_similar_words.shape

(10, 20)

In [46]:
def plot_embedding(X, y, title=None):
    x_min, x_max = np.min(X, 0), np.max(X, 0)
    X = (X - x_min) / (x_max - x_min)

    plt.figure()
#     plt.figure(figsize=(300, 300))
    ax = plt.subplot(111)
    for i in range(X.shape[0]):
        plt.text(X[i, 0], X[i, 1], str(y[i]),
                 fontdict={'size': 9})
        
    plt.xticks([]), plt.yticks([])
    if title is not None:
        plt.title(title)

In [171]:
def plot_20_similar_words(X_tsne, y, word):
    idx = list(vocab_processor.transform([word]))[0][0]
    dist = euclidean_distances(X_tsne[idx:idx+1], X_tsne)
    most_similar_words = dist.argsort(axis=1)[:,:20]
    Y = [words[most_similar_words[0][j]] for j in range(20)]
    X = X_tsne[most_similar_words[0]]
    plot_embedding(X, Y,title= "t-SNE embedding - %s" % word)

In [ ]:
from sklearn import manifold
tsne = manifold.TSNE(n_components=2, init='pca', random_state=0)
X_tsne = tsne.fit_transform(embeddings)

In [ ]:
for word in ['life', 'market', 'stanford', 'trump', 'public']:
    plot_20_similar_words(X_tsne, y, word)

In [ ]:
list(vocab_processor.transform(['cell']))[0][0]